In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import math
import subprocess
import re
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Word2Vec

In [2]:
files_webshell = os.listdir("./data/webshell/")
files_common = os.listdir("./data/normal/")

In [3]:
print("normal file:",len(files_common))
print("webshell file:",len(files_webshell))
print(files_common[1:5])

normal file: 412
webshell file: 1634
['305.php', '306.php', '400.php', '401.php']


In [4]:
#打上文件的label标签，如果是webshell文件，则置label=1 否则 label=0
labels_webshell = []
labels_common = []
for i in range(0,len(files_webshell)):
    labels_webshell.append(1)
for i in range(0,len(files_common)):
    labels_common.append(0)


for i in range(0,len(files_webshell)):
    files_webshell[i] = "D:/week11-webshell-detect/data/webshell/" + files_webshell[i]
for i in range(0,len(files_common)):
    files_common[i] = "D:/week11-webshell-detect/data/normal/" + files_common[i]

In [5]:
files = files_webshell + files_common#将webshell和normal file进行合并
labels = labels_webshell + labels_common

datadict = {'file':files,'label':labels}
df = pd.DataFrame(datadict,columns=['file','label'])
print(df.shape)
print(df.head())

(2046, 2)
                                                file  label
0  D:/week11-webshell-detect/data/webshell/003928...      1
1  D:/week11-webshell-detect/data/webshell/006620...      1
2  D:/week11-webshell-detect/data/webshell/009752...      1
3  D:/week11-webshell-detect/data/webshell/00d0f1...      1
4  D:/week11-webshell-detect/data/webshell/0105d0...      1


In [6]:
#获取php中的opcode。
def getopcode(x):
    try:
        cmd = "php -dvld.active=1 -dvld.execute=0 " + str(x)
        output =  subprocess.getoutput(cmd)
        oplist = re.findall(r'\s(\b[A-Z_]+\b)\s',output)
        #print(oplist)
        #print(str(x))
        return oplist
    except:
        #print("error" + str(x))
        return None

In [7]:
#求文件的信息熵
def getfileshan(x):
    #length = 0
    word = {}
    p = 0
    sum = 0
    with open(x,'r',encoding='ISO-8859-1') as f:
        content = f.readlines()
        for i in content:
            for j in i:
                if j != '\n' and j != ' ':
                    if j not in word.keys():
                        word[j] = 1
                    else:
                        word[j] = word[j] + 1
                else:
                    pass
        f.close()
    for i in word.keys():
        sum = sum + word[i]
    for i in word.keys():
        p = p - float(word[i])/sum * math.log(float(word[i])/sum,2)
    return p

In [8]:
#获取词向量
def getw2v(opc_list,label_list):
    #print(label_list[0:10])
    stop = []
    w2v_list = []
    for i in range(0,7789):
        try:
            #print(label_list[0:10])
            tmp = []
            name = opc_list[i]
            #print(name[0])
            for j in range(0,len(name)):
                tmp.append(name[j])
            w2v_list.append(tmp)
        except:
            pass

    model = Word2Vec(w2v_list, min_count = 1)
    #print (model._vocabulary)
    model.wv.save_word2vec_format('word2vec.txt',binary=False) 
    #return 0
    #print model['a']
    label_vect = []
    wv_vect = []
    for i in range(0,7789):
        try:
            #print(i)
            name = opc_list[i]
            tmp = []
            vect = []
            for j in range(0,len(name)):
                if name[j] in stop:
                    continue
                tmp.append(model[name[j]])
                if j >= 99:
                    break
            if len(tmp) < 100:
                for k in range(0,100-len(tmp)):
                    tmp.append([0]*100)
            vect = np.vstack((x for x in tmp))
            wv_vect.append(vect)
            label_vect.append(label_list[i])
            #if i ==100000:
            #   break
        except:
            pass
    wv_vect = np.array(wv_vect)
    label_vect = np.array(label_vect)
    return wv_vect,label_vect

In [9]:
df['opcode'] = df['file'].map(lambda x:getopcode(x))
df = df.dropna()
#print(getopcode())
print(df.head())

                                                file  label  \
0  D:/week11-webshell-detect/data/webshell/003928...      1   
1  D:/week11-webshell-detect/data/webshell/006620...      1   
2  D:/week11-webshell-detect/data/webshell/009752...      1   
3  D:/week11-webshell-detect/data/webshell/00d0f1...      1   
4  D:/week11-webshell-detect/data/webshell/0105d0...      1   

                                              opcode  
0                                                 []  
1  [E, O, E, INIT_FCALL, INIT_FCALL, SEND_VAL, SE...  
2  [E, O, E, ASSIGN, ASSIGN, ASSIGN, ASSIGN, ASSI...  
3  [E, O, E, INIT_FCALL, FETCH_R, FETCH_DIM_R, SE...  
4  [E, O, E, ASSIGN, INIT_FCALL, FETCH_DIM_R, FET...  


In [10]:
from sklearn.utils import shuffle
df = shuffle(df)
print(df.shape)
w2v_word_list,label_list = getw2v(df['opcode'],df['label'])
print(w2v_word_list[1:5])
print(label_list[1:5])

(2029, 3)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[[[ 1.20605826 -0.70300782  1.28268433 ...  1.6583811  -0.76650095
    0.67054927]
  [-0.06511515  1.30076301  1.71048164 ... -0.39202583  0.16971405
   -0.43985051]
  [ 1.20605826 -0.70300782  1.28268433 ...  1.6583811  -0.76650095
    0.67054927]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 1.20605826 -0.70300782  1.28268433 ...  1.6583811  -0.76650095
    0.67054927]
  [-0.06511515  1.30076301  1.71048164 ... -0.39202583  0.16971405
   -0.43985051]
  [ 1.20605826 -0.70300782  1.28268433 ...  1.6583811  -0.76650095
    0.67054927]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 1.20605826 -0.70300782  1.282684

In [11]:
from sklearn.model_selection import train_test_split
x_train = np.concatenate((w2v_word_list[0:500],w2v_word_list[1500:2000]))
y_train = np.concatenate((label_list[0:500] , label_list[1500:2000]))
x_test = np.concatenate((w2v_word_list[500:1500] , w2v_word_list[2000:]))
y_test = np.concatenate((label_list[500:1500] , label_list[2000:]))

In [12]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding
from keras.layers import LSTM
import pickle
model = Sequential()
#model.add(Embedding())
model.add(LSTM(128,dropout = 0.2,recurrent_dropout = 0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

pickle_file = open('webshell.model','wb')
pickle.dump(model,pickle_file)
pickle_file.close()

Using TensorFlow backend.
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:544: FutureWarning: Passing (type, 1) or '1type' 

In [13]:
print ('start training....')
model.fit(x_train,y_train,epochs = 16,batch_size = 32)
print (' evaling....')
score,acc = model.evaluate(x_test,y_test)
print ("LSTM-score:",score)
print("LSTM-accuracy:",acc)

start training....
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5580 - acc: 0.7560
Epoch 2/16
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3594 - acc: 0.8730
Epoch 3/16
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3076 - acc: 0.8910
Epoch 4/16
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3190 - acc: 0.8930
Epoch 5/16
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2659 - acc: 0.9140A: 0s - loss: 0.2642 - acc: 0.9
Epoch 6/16
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2333 - acc: 0.9150
Epoch 7/16
1000/1000 [==============================] - 4s 4ms/step - loss: 0.1860 - acc: 0.9370
Epoch 8/16
1000/1000 [==============================] - 4s 4ms